In [1]:
import cptac
import numpy as np
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plot_utils as p
from matplotlib_venn import venn2, venn2_circles, venn2_unweighted
from matplotlib_venn import venn3, venn3_circles
from matplotlib import pyplot as plt
%matplotlib inline

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
en = cptac.Endometrial()
hn = cptac.Hnscc()
l = cptac.Luad()
o = cptac.Ovarian()
c = cptac.Ccrcc()
col = cptac.Colon()
g = cptac.Gbm()
b = cptac.Brca()
lscc = cptac.Lscc()

cptac warning: The HNSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 2)


cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 7)


cptac warning: The LSCC data is currently strictly reserved for CPTAC investigators. Otherwise, you are not authorized to access these data. Additionally, even after these data become publicly available, they will be subject to a publication embargo (see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details). (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 9)


In [3]:
def merge_data_frames(cancer):
    Pik = cancer.get_genotype_all_vars("PIK3CA")
    missense = Pik.loc[Pik['Mutation'] == "Missense_Mutation"]
    hotspot = missense[(missense.Location.str.contains('E542K'))|(missense.Location.str.contains('E545K'))|(missense.Location.str.contains('H1047R'))]

    egfr = cancer.get_genotype_all_vars("EGFR")
    amp = egfr.loc[egfr['Mutation'] == "Amplification"]
    pten = cancer.get_genotype_all_vars("PTEN")
    pten_Del = pten.loc[pten['Mutation'] == "Deletion" ]
    #pten_Del = pten.loc[pten['Mutation'] == "Frame_Shift_Del" ]
    brca_all_mut = pd.merge(pten_Del, amp, on="Patient_ID", how = "outer")
    brca_all_mut = brca_all_mut.rename(columns={"Mutation_x": "PTEN_Mutation","Mutation_y":"EGFR_Mutation"  })
    brca_all_mut= pd.merge(brca_all_mut, hotspot, on = "Patient_ID", how = "outer")
    brca_all_mut = brca_all_mut.rename(columns={"Mutation": "PIK3CA_Hotspot"})
    brca_all_mut = brca_all_mut[["EGFR_Mutation","PTEN_Mutation","PIK3CA_Hotspot"]]
    return brca_all_mut
    

In [4]:
def merge_data_frames2(cancer):
    egfr = cancer.get_genotype_all_vars("EGFR")
    amp = egfr.loc[egfr['Mutation'] == "Amplification"]
    pten = cancer.get_genotype_all_vars("PTEN")
    pten_Del = pten.loc[pten['Mutation'] == "Deletion" ]
    brca_all_mut = pd.merge(pten_Del, amp, on="Patient_ID", how = "outer")
    brca_all_mut = brca_all_mut.rename(columns={"Mutation_x": "PTEN_Mutation","Mutation_y":"EGFR_Mutation"  })
    
    brca_all_mut = brca_all_mut[["EGFR_Mutation","PTEN_Mutation"]]
    return brca_all_mut

In [5]:
def HasMutation(row):
    hasMut1 = False
    hasMut2 = False
    for item in row:
        if pd.isnull(item):
            continue
        if item == "Amplification":
            hasMut1 = True
        if item == "Deletion":
            hasMut2 = True
            
    if hasMut1 & hasMut2:
        return "Both"
    if hasMut1 & (hasMut2 == False):
        return "EGFR_mut"
    if (hasMut1 == False) & hasMut2:
        return "Pten_mut"
    return False

In [6]:
def All3(row):
    hasMut1 = False
    hasMut2 = False
    hasMut3 = False
    for item in row:
        if pd.isnull(item):
            continue
        if item == "Amplification":
            hasMut1 = True
        if item == "Deletion":
            hasMut2 = True
        if item == "Missense_Mutation":
            hasMut3 = True
            
    if hasMut1 & hasMut2 & hasMut3:
        return True
    return False

In [7]:
def EGFR_PTEN(row):
    hasMut1 = False
    hasMut2 = False
    hasMut3 = False
    for item in row:
        if pd.isnull(item):
            continue
        if item == "Amplification":
            hasMut1 = True
        elif item == "Deletion":
            hasMut2 = True
        
        elif item == "Missense_Mutation":
            hasMut3 = True
            
    if hasMut1 & hasMut2 & (hasMut3 == False):
        return True
    return False

In [8]:
def EGFR_PTEN(row):
    hasMut1 = False
    hasMut2 = False
    hasMut3 = False
    for item in row:
        if pd.isnull(item):
            continue
        if item == "Amplification":
            hasMut1 = True
        elif item == "Deletion":
            hasMut2 = True
        
        elif item == "Missense_Mutation":
            hasMut3 = True
            
    if hasMut1 & hasMut2 & (hasMut3 == False):
        return True
    return False

In [9]:
def EGFR_PIK3CA(row):
    hasMut1 = False
    hasMut2 = False
    hasMut3 = False
    for item in row:
        if pd.isnull(item):
            continue
        if item == "Amplification":
            hasMut1 = True
        elif item == "Deletion":
            hasMut2 = True
        
        elif item == "Missense_Mutation":
            hasMut3 = True
            
    if hasMut1 & (hasMut2 == False) & hasMut3:
        return True
    return False

In [10]:
def PTEN_PIK3CA(row):
    hasMut1 = False
    hasMut2 = False
    hasMut3 = False
    for item in row:
        if pd.isnull(item):
            continue
        if item == "Amplification":
            hasMut1 = True
        elif item == "Deletion":
            hasMut2 = True
        
        elif item == "Missense_Mutation":
            hasMut3 = True
            
    if (hasMut1 == False) & hasMut2 & hasMut3:
        return True
    return False

# Brca

In [11]:
brca = merge_data_frames(b)
brca["All3"] = brca.apply(All3, axis = 1)
brca["EGFR_PTEN"] = brca.apply(EGFR_PTEN, axis = 1)
brca["EGFR_PIK3CA"] = brca.apply(EGFR_PIK3CA, axis = 1)
brca["PTEN_PIK3CA"] = brca.apply(PTEN_PIK3CA, axis = 1)
data = {'All_3':  [sum(brca["All3"])],
        'EGFR_PTEN': [sum(brca["EGFR_PTEN"])],
        'EGFR_PIK3CA': [sum(brca["EGFR_PIK3CA"])],
        'PTEN_PIK3CA': [sum(brca["PTEN_PIK3CA"])],
        "Total_Samples" :[len(brca)],
        "Cancer_Type" : ["BRCA"]
        }

df = pd.DataFrame (data, columns = ["Cancer_Type",'All_3','EGFR_PTEN','EGFR_PIK3CA',"PTEN_PIK3CA","Total_Samples"])


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the PIK3CA gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 394)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 118 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 394)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 113 samples for the PTEN gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 394)


# Endometrial

In [45]:
endo = merge_data_frames(en)
endo["All3"] = endo.apply(All3, axis = 1)
endo["EGFR_PTEN"] = endo.apply(EGFR_PTEN, axis = 1)
endo["EGFR_PIK3CA"] = endo.apply(EGFR_PIK3CA, axis = 1)
endo["PTEN_PIK3CA"] = endo.apply(PTEN_PIK3CA, axis = 1)
data = {'All_3':  [sum(endo["All3"])],
        'EGFR_PTEN': [sum(endo["EGFR_PTEN"])],
        'EGFR_PIK3CA': [sum(endo["EGFR_PIK3CA"])],
        'PTEN_PIK3CA': [sum(endo["PTEN_PIK3CA"])],
        "Total_Samples" :[len(endo)],
        "Cancer_Type" : ["Endo"]
        }

df2 = pd.DataFrame (data, columns = ["Cancer_Type",'All_3','EGFR_PTEN','EGFR_PIK3CA',"PTEN_PIK3CA","Total_Samples"])
df = df.append(df2)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 48 samples for the PIK3CA gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 91 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 20 samples for the PTEN gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


# Colon

In [46]:
colon = merge_data_frames(col)
colon.head()
colon["All3"] = colon.apply(All3, axis = 1)
colon["EGFR_PTEN"] = colon.apply(EGFR_PTEN, axis = 1)
colon["EGFR_PIK3CA"] = colon.apply(EGFR_PIK3CA, axis = 1)
colon["PTEN_PIK3CA"] = colon.apply(PTEN_PIK3CA, axis = 1)
data = {'All_3':  [sum(colon["All3"])],
        'EGFR_PTEN': [sum(colon["EGFR_PTEN"])],
        'EGFR_PIK3CA': [sum(colon["EGFR_PIK3CA"])],
        'PTEN_PIK3CA': [sum(colon["PTEN_PIK3CA"])],
        "Total_Samples" :[len(colon)],
        "Cancer_Type" : ["Colon"]
        }

df2 = pd.DataFrame (data, columns = ["Cancer_Type",'All_3','EGFR_PTEN','EGFR_PIK3CA',"PTEN_PIK3CA","Total_Samples"])
df = df.append(df2)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 82 samples for the PIK3CA gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 103 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 05CO045 (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: 

# GBM

In [19]:
gbm = merge_data_frames2(g)
gbm["Both"] = gbm.apply(HasMutation, axis = 1)

gbm


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 83 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-01834 (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 72 samples for the PTEN gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: C3L-01834 (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


Name,EGFR_Mutation,PTEN_Mutation,Both
Patient_ID,,,
C3L-00104,Amplification,Deletion,True
C3L-00365,NaN,Deletion,Pten_mut
C3L-00674,Amplification,Deletion,True
C3L-00677,Amplification,Deletion,True
C3L-01040,Amplification,Deletion,True
...,...,...,...
C3L-03260,Amplification,NaN,EGFR_mut
C3L-03392,Amplification,NaN,EGFR_mut
C3N-02784,Amplification,NaN,EGFR_mut


# Luad

In [11]:
luad = merge_data_frames2(l)
luad["Both"] = luad.apply(HasMutation, axis = 1)
data = {
        'EGFR_PTEN': [sum(luad["Both"])],
        "Total_Samples" :[len(luad)],
        "Cancer_Type" : ["Luad"]
        }

df2 = pd.DataFrame (data, columns = ["Cancer_Type",'EGFR_PTEN',"Total_Samples"])
df = df.append(df2)




cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 69 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 313)


TypeError: unsupported operand type(s) for +: 'int' and 'str'

# LSCC

In [49]:
Lscc = merge_data_frames2(lscc)
Lscc["Both"] = Lscc.apply(HasMutation, axis = 1)
data = {
        'EGFR_PTEN': [sum(Lscc["Both"])],
        "Total_Samples" :[len(Lscc)],
        "Cancer_Type" : ["Lscc"]
        }

df2 = pd.DataFrame (data, columns = ["Cancer_Type",'EGFR_PTEN',"Total_Samples"])
df = df.append(df2)


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 106 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 99 samples for the PTEN gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)


# Ovarian

In [51]:
Ovar = merge_data_frames2(o)
Ovar["Both"] = Ovar.apply(HasMutation, axis = 1)
data = {
        'EGFR_PTEN': [sum(Ovar["Both"])],
        "Total_Samples" :[len(Ovar)],
        "Cancer_Type" : ["Ovarian"]
        }

df2 = pd.DataFrame (data, columns = ["Cancer_Type",'EGFR_PTEN',"Total_Samples"])
df = df.append(df2)
df.set_index("Cancer_Type")


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 101 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 01OV029, 02OV015 (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 98 samples for the PTEN gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)
cptac warning: CNV data was not found for the following samples, so CNV data columns were filled with NaN for these samples: 01OV029, 02OV015 (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/cptac/dataset.py, line 383)

,All_3,EGFR_PTEN,EGFR_PIK3CA,PTEN_PIK3CA,Total_Samples
Cancer_Type,,,,,
BRCA,1.0,12,3.0,1.0,55
Endo,0.0,0,1.0,0.0,22
Colon,0.0,18,0.0,0.0,72
GBM,NaN,61,NaN,NaN,86
Luad,NaN,7,NaN,NaN,36
Lscc,NaN,9,NaN,NaN,47
Ovarian,NaN,7,NaN,NaN,42
Ovarian,NaN,7,NaN,NaN,42


NameError: name 'cancer' is not defined